# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
input_data_file = "../output_data/cities.csv"

city_data_df = pd.read_csv(input_data_file)
city_data_df.drop(columns=['Unnamed: 0'], inplace=True)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Set the locations
locations = city_data_df[["Lat", "Lon"]].astype(float)

In [5]:
# Set the weight
humidity = city_data_df['Humidity'].astype(float)

In [6]:
fig = gmaps.figure(center=locations.iloc[3,0:2].tolist(), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

plt.savefig("../Images/humidity_map.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
good_cities_data_df = city_data_df.loc[((city_data_df['Max Temp'] >80) & (city_data_df['Max Temp'] < 90) & (city_data_df['Humidity'] < 50) & (city_data_df['Cloudiness'] < 50) & (city_data_df['Wind Speed'] < 10))]

In [8]:
good_cities_data_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,Carinhanha,-14.3047,-43.7650,81.70,31,1,3.18,BR,1659645956
201,Saint Anthony,45.0205,-93.2180,88.14,39,0,6.91,US,1659645920
392,Menomonie,44.8755,-91.9193,83.73,44,0,5.75,US,1659646070
397,Ocosingo,17.0667,-92.2500,83.53,33,9,6.67,MX,1659646071
480,Abu Samrah,35.3029,37.1841,85.66,38,0,6.26,SY,1659646099
481,Capinópolis,-18.6819,-49.5697,82.72,26,39,1.70,BR,1659646099
531,Korla,41.7597,86.1469,81.79,34,1,3.40,CN,1659646117
549,Kaka,37.3482,59.6143,83.53,20,0,3.58,TM,1659646122


In [9]:
# Cities with missing data were rejected in the original API calls
# Otherwise I'd use good_cities_data_df = good_cities_data_df.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = good_cities_data_df.copy()

In [11]:
hotel_df['Hotel Name'] = ''

In [12]:
hotel_df.reset_index(inplace=True, drop=True)

In [13]:
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Carinhanha,-14.3047,-43.7650,81.70,31,1,3.18,BR,1659645956,
1,Saint Anthony,45.0205,-93.2180,88.14,39,0,6.91,US,1659645920,
2,Menomonie,44.8755,-91.9193,83.73,44,0,5.75,US,1659646070,
3,Ocosingo,17.0667,-92.2500,83.53,33,9,6.67,MX,1659646071,
4,Abu Samrah,35.3029,37.1841,85.66,38,0,6.26,SY,1659646099,
5,Capinópolis,-18.6819,-49.5697,82.72,26,39,1.70,BR,1659646099,
6,Korla,41.7597,86.1469,81.79,34,1,3.40,CN,1659646117,
7,Kaka,37.3482,59.6143,83.53,20,0,3.58,TM,1659646122,


In [14]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_radius = 5000

In [15]:
#use iterrows to iterate through pandas dataframe
params = {
    "radius": target_radius,
    "type": "lodging",    
    "key": g_key,
    }

for index, row in hotel_df.iterrows():
    # assemble url and make API request
    lat = hotel_df.iloc[index,1]
    lon = hotel_df.iloc[index,2]
    
    params["location"]=f"{lat},{lon}"

    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in { hotel_df.iloc[ index, 0 ] } is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']       
    except (KeyError, IndexError):
        print(results)
        
    print("------------")

Closest hotel in Carinhanha is Hotel União.
------------
Closest hotel in Saint Anthony is Holiday Inn Express Roseville-St. Paul, an IHG Hotel.
------------
Closest hotel in Menomonie is Best Western Plus Menomonie Inn & Suites.
------------
Closest hotel in Ocosingo is Chabek lumil.
------------
[]
------------
Closest hotel in Capinópolis is Tropical Hotel Capinópolis.
------------
Closest hotel in Korla is Gulisitan Hotel.
------------
[]
------------


In [16]:
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Carinhanha,-14.3047,-43.7650,81.70,31,1,3.18,BR,1659645956,Hotel União
1,Saint Anthony,45.0205,-93.2180,88.14,39,0,6.91,US,1659645920,"Holiday Inn Express Roseville-St. Paul, an IHG..."
2,Menomonie,44.8755,-91.9193,83.73,44,0,5.75,US,1659646070,Best Western Plus Menomonie Inn & Suites
3,Ocosingo,17.0667,-92.2500,83.53,33,9,6.67,MX,1659646071,Chabek lumil
4,Abu Samrah,35.3029,37.1841,85.66,38,0,6.26,SY,1659646099,
5,Capinópolis,-18.6819,-49.5697,82.72,26,39,1.70,BR,1659646099,Tropical Hotel Capinópolis
6,Korla,41.7597,86.1469,81.79,34,1,3.40,CN,1659646117,Gulisitan Hotel
7,Kaka,37.3482,59.6143,83.53,20,0,3.58,TM,1659646122,


In [17]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

In [18]:
hotel_df.dropna(inplace=True)

In [19]:
hotel_df.reset_index(inplace=True, drop=True)

In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [21]:
humidity = hotel_df['Humidity']

In [22]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[['Lat', 'Lon']]

# Display figure
fig = gmaps.figure(center=locations.iloc[3,0:2].tolist(), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig.add_layer(heat_layer)
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Hotel {hotel_df.iloc[index,9]} in {hotel_df.iloc[index,0]}, {hotel_df.iloc[index,7]}" for index in hotel_df.index])
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>